In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from getpass import getpass
import time
import pandas as pd
from PIL import Image
import chromedriver_autoinstaller as chromedriver
from webdriver_manager.chrome import ChromeDriverManager
chromedriver.install()
from selenium.webdriver.chrome.service import Service
options = Options()
options.add_argument("start-maximized")
from IPython.display import display, HTML
from tabulate import tabulate,tabulate_formats
pd.set_option('display.colheader_justify', 'center')
import xlsxwriter
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles.alignment import Alignment
from openpyxl.styles import Alignment
import xlrd
from selenium.webdriver.common.by import By
# https://xlsxwriter.readthedocs.io/worksheet.html

In [2]:
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install())) 



INFO:WDM:

====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
Current google-chrome version is 101.0.4951
INFO:WDM:Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
INFO:WDM:Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
INFO:WDM:Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\aakas\.wdm\drivers\chromedriver\win32\101.0.4951.41]
INFO:WDM:Driver has been saved in cache [C:\Users\aakas\.wdm\drivers\chromedriver\win32\101.0.4951.41]


In [7]:
url = "http://jntuhsit.org/"
driver.get(url)
driver.find_element(By.XPATH,"//a[@href='http://14.139.82.140']").click()
driver.switch_to.window(driver.window_handles[1])
time.sleep(2)

In [10]:
# a = ['21031F0005']
a = [f"21031F00{i:02d}" for i in range(1,31)]
All = {}
for i in a:
    driver.find_element(By.ID,'uid').send_keys(i)
    if i=="21031F0005":
        driver.find_element(By.ID,'pwd').send_keys(getpass(f"{i} Enter Your Password : "))
    else:
        driver.find_element(By.ID,'pwd').send_keys("sit123")
    driver.find_element(By.CSS_SELECTOR,"input[type='submit']").click()
    if driver.find_element(By.XPATH,".//div[contains(@class,'alert')]").text == '×\n  Failed !  Invalid LoginID or Password....':
        continue
    else:
        time.sleep(1)
        driver.find_element(By.XPATH,"//div[@id='myNavBar']/ul/li[4]").click()
        time.sleep(1)
        driver.find_element(By.XPATH,"//a[@href='viewResults']").click()
        time.sleep(1)
        if driver.find_element(By.XPATH,".//div[contains(@class,'alert')]").text == '×\n  Failed !  Results records not found.':
            driver.find_element(By.XPATH,"//a[@href='logout']").click()
            continue
        else:
            driver.find_element(By.XPATH,"//select[@id='ecode']/option[text()='M.C.A. I Year I Semester (R20) (Reg) - April, 2022']").click()
            time.sleep(1)
            driver.find_element(By.XPATH,".//button[contains(@class,'btn')]").click()
            time.sleep(1)
            Result = driver.find_element(By.XPATH,'//div[@id="resBlock"]//table').text.split("\n")
    All[i] = Result
    driver.find_element(By.XPATH,"//a[@href='logout']").click()
print(driver.find_element(By.CLASS_NAME,'alert-success').text.strip(".")[2:40])
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()

21031F0005 Enter Your Password : ········
  Success !  Successfully Logged out


In [3]:
def Grade_Table(roll):
    global columns 
    columns = ["S.No","SubjectCode","SubjectName","Grade","GradePoint","Credits","Result"]
    sln = [i+1 for i in range(len(All[roll][10:-1]))]
    D = {}
    for i in columns:
        D[i] = []
    D[columns[0]] = sln
    D[columns[1]] = [All[roll][10:-1][i].split()[0] for i in range(len(All[roll][10:-1]))]
    D[columns[2]] = [' '.join(All[roll][10:-1][i].split()[1:-4]) for i in range(len(All[roll][10:-1]))]
    D[columns[3]] = [All[roll][10:-1][i].split()[-4] for i in range(len(All[roll][10:-1]))]
    D[columns[4]] = [All[roll][10:-1][i].split()[-3] for i in range(len(All[roll][10:-1]))]
    D[columns[5]] = [All[roll][10:-1][i].split()[-2] for i in range(len(All[roll][10:-1]))]
    D[columns[6]] = [All[roll][10:-1][i].split()[-1] for i in range(len(All[roll][10:-1]))]
    df = pd.DataFrame(D)
    df[['GradePoint', 'Credits']] = df[['GradePoint', 'Credits']].astype('int64')
    return df

In [4]:
All_Results = pd.DataFrame(columns=['RollNumber','Name','SGPA'])
All_Results['RollNumber'] = [list(All.keys())[i] for i in range(len(list(All.keys())))]
All_Results['Name']       = [All[list(All.keys())[i]][7][6:] for i in range(len(list(All.keys())))]
All_Results['SGPA']       = [round(sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"]),2) for i in list(All.keys())]
HTML(All_Results.to_html(index=False))

RollNumber,Name,SGPA
21031F0002,DADYREDDY DHEERAJ KUMAR REDDY,7.10
21031F0004,GANGAPURAM AKHILA,8.14
21031F0005,AAKASH NADUPALLI,9.00
21031F0007,RENUKUNTLA SHIVA RAMA KRISHNA,8.14
21031F0008,VEMURI VIDYA MADHURI,9.24
21031F0009,BOLLIPALLI AKANKSHA,9.05
21031F0010,CHINTHA CHANDANA,8.19
21031F0012,KARNE RAMYA,7.62
21031F0014,M HARSHITH,7.24
21031F0015,BATTIKADI SREEDIVYA,8.57


In [5]:
def Check_Result(roll):
    if roll=='05':
        display(HTML("<div style='position: absolute;top: 200px; right: 80px; width: 200px; text-align:right;'><img width=80,height=70 src='Aakash_New_PassportPhoto.jpg'></div>"))
    roll = '21031F00'+roll
    l= [400,505,400,465,475,400,170,170,170]
    t =[55,88,95,115,140,185,230,250,272]
    h = ['h3','h7','h4','h4','h6','I','I','I','I']
    s = [18,13,16,16,14,17,14,14,14]
    c = ['#f7118b','#0a21a8','#0a21a8','#0a21a8','Red','Black','Black','Black','Black']
    display(HTML("<div style='position: absolute;top: 80px; left: 180px; width: 200px; text-align:left;'><img width=130,height=50 src='Jntuh logo.png'></div>"))
    for i,j,k,m,n,o in zip(range(len(All['21031F0005'][:9])),range(len(t)),range(len(l)),range(len(h)),range(len(s)),range(len(c))):
        display(HTML("<div style='position: absolute;top: {vl}px; left: {lf}px; width: 500px; text-align:left;'><{hd} style='font-family:Arial;color:{cr};font-size:{st}px;'>{Tx}</{hd}></div>".format(Tx = All[roll][:9][i],vl=t[j],lf=l[k],hd=h[m],st=s[n],cr=c[o])))    
    print('\n'*10)
#     display(tabulate(Grade_Table(roll),showindex=False,headers =columns,tablefmt=tabulate_formats[-2]))
    display(HTML(Grade_Table(roll).to_html(index=False)))
    print()
#     print(' '*50,'SGPA : ',SGPA(roll))
    print(' '*50,'SGPA : ',sum(Grade_Table('21031F0005')["GradePoint"] * Grade_Table('21031F0005')["Credits"])/sum(Grade_Table('21031F0005')["Credits"]))

In [6]:
Check_Result(input("Enter Last Two Digits of your RollNumber : "))

Enter Last Two Digits of your RollNumber : 05


S.No,SubjectCode,SubjectName,Grade,GradePoint,Credits,Result
1,MC110,MATHEMATICS FOR COMPUTER SCIENCE,A,8,3,P
2,MC120,COMPUTER ARCHITECTURE & ORGANIZATION,A+,9,3,P
3,MC130,OBJECT ORIENTED PROGRAMMING THROUGH C++,A+,9,3,P
4,MC140,OPERATING SYSTEMS,A+,9,3,P
5,MC150,DATABASE SYSTEMS,A,8,3,P
6,MC160,COMPUTER ARCHITECTURE & ORGANIZATION & OPERATING SYSTEMS LAB,O,10,2,P
7,MC170,OBJECT ORIENTED PROGRAMMING THROUGH C++ LAB,O,10,2,P
8,MC180,DATABASE SYSTEMS LAB,O,10,2,P
9,MC19A,PROFESSIONAL COMMUNICATION SKILLS,A,8,0,P



                                                   SGPA :  9.0


In [112]:
Ex = {}
for i in list(All.keys()):
    k =pd.DataFrame(All[i][0:6])
    l =pd.DataFrame(All[i][6:9])
    m =Grade_Table(i)
    n =pd.DataFrame(data=[f'SGPA  :  {round(sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"]),2)}']).transpose()
#     pd.DataFrame(data=['SGPA',round(sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"]),2)]).transpose()
    dfs = [k,l,m,n]
    Ex[i] = dfs

In [113]:
roll = '21031F0005'
a =pd.DataFrame(All[roll][0:6])
b =pd.DataFrame(All[roll][6:9])
c =pd.DataFrame(columns).transpose()
d =Grade_Table(roll)
e =pd.DataFrame(data=[f'SGPA  : {round(sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"]),2)}']).transpose()
# pd.DataFrame(data=['SGPA',round(sum(Grade_Table(roll)["GradePoint"] * Grade_Table(roll)["Credits"])/sum(Grade_Table(roll)["Credits"]),2)]).transpose()
dfs = [a,b,c,d,e]
startrow = 2
startcol = [5,5,3,3,5]
with pd.ExcelWriter('MCA_1st_Sem_Results.xlsx',engine="xlsxwriter") as writer:
    for (df,c,k) in zip(dfs,startcol,range(len(dfs))):
        if k==2:
            df.to_excel(writer,sheet_name=roll,startcol=c,startrow=startrow,index=False,header=False)
            startrow += 1
        else:
            df.to_excel(writer,sheet_name=roll,startcol=c,startrow=startrow,index=False,header=False)
            startrow += (df.shape[0] + 2)    

In [114]:
All_dfs  = {}
for r in list(All.keys()):
    x = pd.read_excel('MCA_1st_Sem_Results.xlsx')
    x.iloc[1:7,5:6]   =  Ex[r][0]
    x.iloc[9:12,5:6]  =  Ex[r][1]
    x.iloc[15:24,3:11] =  Ex[r][2]
    x.iloc[25:,5:6]   =  Ex[r][3]
    x = x.iloc[0:-1]
    All_dfs[r] = x
All_dfs['All_Results'] = All_Results

In [115]:
with pd.ExcelWriter('MCA_1st_Sem_Results.xlsx',engine="xlsxwriter") as writer:
    for df in All_dfs:
        All_dfs[df].to_excel(writer,sheet_name=df,index=False,header=False)
        if df=='All_Results':
            All_dfs[df].to_excel(writer,sheet_name=df,index=False)
    writer.save()

C:\Users\aakas\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [116]:
for rl in list(All_dfs.keys()):
    wb = load_workbook('MCA_1st_Sem_Results.xlsx')
    if rl!='All_Results':
        ws = wb[rl]
        for (i,j) in zip(['E','F','H'],[14,65,12]):
            ws.column_dimensions[i].width = j
        img = openpyxl.drawing.image.Image('Jntuh Logo.png')
        img.height = 150
        img.width = 170
        ws.add_image(img,'D3')
        if rl =='21031F0005':
            img = openpyxl.drawing.image.Image('Aakash_New_PassportPhoto.jpg')
            img.height = 110
            img.width = 110
            ws.add_image(img,'G8')
            ws.column_dimensions['G'].width = 15
            ws.cell(15,7).alignment = Alignment(horizontal='center')
    else:
        ws = wb[rl] 
        for (i,j) in zip(['A','B','C'],[18,50,15]):
            ws.column_dimensions[i].width = j
    for p in [int(i) for i in range(1,11)]:
        for j in [int(i) for i in range(1,31)]:
            ws.cell(j,p).alignment = Alignment(horizontal='center')
        wb.save('MCA_1st_Sem_Results.xlsx')        
wb.close()

In [98]:
# wb = load_workbook('MCA_1st_Sem_Results.xlsx')
# ws = wb.worksheets[0]
# img = openpyxl.drawing.image.Image('Jntuh Logo.png')
# img.height = 130
# img.width = 150
# ws.add_image(img,'E3')
# wb.save('MCA_1st_Sem_Results.xlsx')

In [111]:
# wb = load_workbook('MCA_1st_Sem_Results.xlsx')
# ws = wb.worksheets[1]
# img = openpyxl.drawing.image.Image('Aakash_New_PassportPhoto.jpg')
# img.height = 100
# img.width = 130
# ws.add_image(img,'G9')
# ws.column_dimensions['G'].width = 8
# ws.column_dimensions['H'].width = 10
# ws.cell(15,7).alignment = Alignment(horizontal='center')
# # ws.cell(15,7).alignment = Alignment(horizontal='center')
# wb.save('MCA_1st_Sem_Results.xlsx')

In [82]:
# wb = load_workbook('MCA_1st_Sem_Results.xlsx')
# ws = wb.worksheets[0]
# img = openpyxl.drawing.image.Image('Aakash_New_PassportPhoto.jpg')
# img.height = 100
# img.width = 110
# ws.add_image(img,'G8')
# ws.column_dimensions['G'].width = 15
# ws.cell(15,7).alignment = Alignment(horizontal='center')
# wb.save('MCA_1st_Sem_Results.xlsx')

In [86]:
# from openpyxl import Workbook
# from openpyxl.drawing.image import Image
# from openpyxl.drawing.spreadsheet_drawing import AbsoluteAnchor
# from openpyxl.drawing.xdr import XDRPoint2D, XDRPositiveSize2D
# from openpyxl.utils.units import pixels_to_EMU, cm_to_EMU

# wb = load_workbook('MCA_1st_Sem_Results.xlsx')
# ws = wb.worksheets[0]
# img = Image('Aakash_New_PassportPhoto.jpg')

# p2e = pixels_to_EMU

# h, w = img.height, img.width

# position = XDRPoint2D(p2e(500), p2e(500))
# size = XDRPositiveSize2D(p2e(w), p2e(h))

# img.anchor = AbsoluteAnchor(pos=position, ext=size)
# ws.add_image(img) 
# wb.save('MCA_1st_Sem_Results.xlsx')